In [ ]:
# ==============================================================================
# Notebook para Tratamento e Extração da Coluna 'narrativa'
# Saída: um CSV com uma única coluna, pronto para Vertex AI
# ==============================================================================

# --- 1. IMPORTAÇÕES ---
import pandas as pd
import os
from tqdm.auto import tqdm

# Configura o tqdm para funcionar de forma integrada com o pandas
tqdm.pandas()

# --- 2. CONFIGURAÇÃO DOS CAMINHOS ---
PASTA_DE_DADOS = os.path.join('data', 'data_final')
ARQUIVO_CSV_ENTRADA = os.path.join(PASTA_DE_DADOS, 'INFLUD-IDOSOS_TRATADO.csv')
# O nome do arquivo de saída agora reflete seu conteúdo
ARQUIVO_CSV_SAIDA = os.path.join(PASTA_DE_DADOS, 'NARRATIVAS_PARA_VERTEX.csv')

print(f"Arquivo de entrada: {ARQUIVO_CSV_ENTRADA}")
print(f"Arquivo de saída: {ARQUIVO_CSV_SAIDA}")

# --- 3. MAPEAMENTO DE DADOS ---
# (Esta seção é necessária para criar a narrativa corretamente)
SYMPTOM_MAP = {
    'FEBRE': 'febre', 'TOSSE': 'tosse', 'GARGANTA': 'dor de garganta',
    'DISPNEIA': 'dificuldade de respirar (dispneia)', 'DESC_RESP': 'desconforto respiratório',
    'SATURACAO': 'baixa saturação de oxigênio', 'DIARREIA': 'diarreia',
    'VOMITO': 'vômito', 'DOR_ABD': 'dor abdominal', 'FADIGA': 'fadiga',
    'PERD_OLFT': 'perda de olfato', 'PERD_PALA': 'perda de paladar'
}
FACTOR_MAP = {
    'FATOR_RISC': 'possui fator de risco', 'VACINA_COV': 'foi vacinado para covid',
    'VACINA': 'recebeu vacina para influenza'
}

# --- 4. FUNÇÃO DE TRANSFORMAÇÃO ---
def create_narrative_from_row(row):
    try:
        narrativa = f"Registro de um paciente de {int(row['NU_IDADE_N'])} anos."
        fatores = [desc for col, desc in FACTOR_MAP.items() if col in row and row.get(col) == 1]
        if fatores: narrativa += f" O paciente {', '.join(fatores)}."
        sintomas = [desc for col, desc in SYMPTOM_MAP.items() if col in row and row.get(col) == 1]
        if sintomas: narrativa += f" Apresentou os seguintes sintomas: {', '.join(sintomas)}."
        else: narrativa += " Não apresentou os principais sintomas listados."
        if 'diagnostico' in row and pd.notna(row['diagnostico']):
            narrativa += f" O diagnostico final recebido foi {row['diagnostico']}."
        return narrativa
    except (ValueError, TypeError) as e:
        print(f"Erro ao processar linha {row.name}: {e}. Pulando linha.")
        return ""

# --- 5. PROCESSO DE LEITURA E TRANSFORMAÇÃO ---
print("\nIniciando processo de ETL...")
print(f"Lendo arquivo: {ARQUIVO_CSV_ENTRADA}")
try:
    df = pd.read_csv(ARQUIVO_CSV_ENTRADA, sep=',', encoding='latin-1', low_memory=False)
except Exception:
    df = pd.read_csv(ARQUIVO_CSV_ENTRADA, sep=';', encoding='latin-1', low_memory=False)

print("Limpando nomes das colunas...")
df.columns = df.columns.str.strip()

print("Convertendo tipos de dados...")
cols_to_check = list(SYMPTOM_MAP.keys()) + list(FACTOR_MAP.keys())
for col in cols_to_check:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
df['NU_IDADE_N'] = pd.to_numeric(df['NU_IDADE_N'], errors='coerce').fillna(0).astype(int)

print("Gerando narrativas para cada registro...")
df['narrativa'] = df.progress_apply(create_narrative_from_row, axis=1)
df.dropna(subset=['narrativa'], inplace=True) # Remove linhas onde a narrativa falhou
df = df[df['narrativa'] != ""]
print("\nProcesso de transformação concluído.")

# --- 6. CRIAÇÃO DO DATAFRAME FINAL ---
# <--- MUDANÇA PRINCIPAL AQUI
print("\nCriando DataFrame final contendo apenas a coluna 'narrativa'...")
# Usamos colchetes duplos [[]] para garantir que o resultado seja um DataFrame
df_final = df[['narrativa']]
print("DataFrame final criado. Amostra:")
print(df_final.head().to_string())


# --- 7. EXPORTAÇÃO DO RESULTADO ---
print(f"\nExportando {len(df_final)} narrativas para o arquivo de saída...")
# Salva o DataFrame final (df_final) em um novo arquivo CSV
df_final.to_csv(ARQUIVO_CSV_SAIDA, index=False, encoding='utf-8-sig')

print(f"\n--- SUCESSO! ---")
print(f"Arquivo '{ARQUIVO_CSV_SAIDA}' foi criado com sucesso.")
print("Ele contém APENAS a coluna 'narrativa' e está pronto para o Vertex AI Search.")

Arquivo de entrada: data\data_final\INFLUD-IDOSOS_TRATADO.csv
Arquivo de saída: data\data_final\NARRATIVAS_PARA_VERTEX.csv

Iniciando processo de ETL...
Lendo arquivo: data\data_final\INFLUD-IDOSOS_TRATADO.csv
Limpando nomes das colunas...
Convertendo tipos de dados...
Gerando narrativas para cada registro...


  0%|          | 0/2479 [00:00<?, ?it/s]

KeyError: 'diagnóstico'